In [1]:
from pytorch_libraries.utility_dataset import *
from pytorch_libraries.model_CNN import *

from IPython.display import clear_output

from pylab import *
%matplotlib inline
#%qtconsole #--style monokai

# 0. Load dataset and Model

In [109]:
trnset = np.fromfile("../data/MNIST/MNIST_train_data.dat").reshape(-1,785)
tstset = np.fromfile("../data/MNIST/MNIST_test_data.dat" ).reshape(-1,785)
trnset[:,-1] = trnset[:,-1] - 1 # make label 0-9
tstset[:,-1] = tstset[:,-1] - 1 # make label 0-9

model = CNN_MNIST().cuda()
#model = torch.load('../data/model/MNIST_CNN.pt').cuda()
trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize(mean=(0.5,0.5,0.5),
                                                 std=(0.5,0.5,0.5))
                           ])

trnset = tcDataset(trnset,shape=(-1,1,28,28))#,transform=trans)
tstset = tcDataset(tstset,shape=(-1,1,28,28))#,transform=trans)

# 1. Training

In [111]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
batches   = DataLoader(trnset,batch_size=100, shuffle=True)

model.train()
# begin Training
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(batches):
        
        # 1.get the inputs
        inputs = Variable(batch["data"]).cuda()
        labels = Variable(batch["label"]).cuda()
        # 2.zero the parameter gradients
        optimizer.zero_grad()
        # 3.forward + backward + optimize
        outputs = model(inputs)
        loss    = F.nll_loss( outputs, labels )
        loss.backward()
        optimizer.step()
        
        
        # TRAINING DESHBOARD #
        ##################################################################
        running_loss += loss.data[0]
        if i % 100 == 99:    # print every 100 mini-batches
            predicts = (outputs.data).max(1)[1]
            corrects = predicts.eq(labels.data).sum()
            accuracy = corrects/batches.batch_size
            print('[{:5d}, {:5d}] loss: {:5.3f}, acc: {:5.3f}'.
                  format(epoch+1, i+1, running_loss/100, accuracy))
            clear_output(wait=True)
            running_loss = 0.0
        ##################################################################


[   10,   600] loss: 0.051, acc: 0.980


# 2. Test Accuracy

In [ ]:
corrects = 0
model.eval()
model = model.cpu()
for batch in DataLoader(trnset,batch_size=len(trnset)):
    inputs  = Variable(batch["data"])
    labels  = Variable(batch["label"])
    outputs  = model(inputs)
    predicts = (outputs.data).max(1)[1] # get the index of the max log-probability
    corrects = corrects + predicts.eq(labels.data).sum()
    
print("acc:{:5.3f}".format(corrects/len(tstset)))

In [ ]:
torch.save(model.cpu(),'../data/model/MNIST_CNN.pt') 